<a href="https://colab.research.google.com/github/kai0127/Jupyter-Notebooks/blob/master/intro_to_ml_hw4_recitation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Decision Tree

First part of the homework required you to read the data and fit a decision tree by using sklearn libraries.
We will use: 
1. `sklearn.tree.DecisionTreeClassifier` to fit tree to the data.
2. We will load the data in https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data by using pandas library. 
3. Finally we'll fit a tree to the data at each iteration.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


In [ ]:
random_seeds = [i for i in range(20)]
criterion = 'gini' #gini, entropy, log_loss
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data', header=None, index_col=57) # Number of attributes in the dataset is 57

In [ ]:
# Create the training loop
# For each seed we will be splitting the data, fit the tree and get the accuracy
accuracies = []
for random_seed in random_seeds: 
    # Get the data and fit a classifier
    X_train, X_test, y_train, y_test = train_test_split(data, data.index.values, test_size=0.3, random_state=random_seed)
    classifier = DecisionTreeClassifier(splitter='random', random_state=random_seed)
    classifier.fit(X_train, y_train)

    # Predict
    y_pred = classifier.predict(X_test)

    # Get the accuracy score
    accuracies.append(accuracy_score(y_test, y_pred))

In [ ]:
# Get the maximum accuracy
accuracies = np.array(accuracies)
print(f'Decision Tree Max accuracy: {accuracies.max()}\tCriterion: {criterion}')

Decision Tree Max accuracy: 0.9254163649529327 with criterion: gini


## Random Forest

Now we will again use already implemented random forest classifiers with different seeds each.
Very similar to decision trees only with multiple estimators and taking their advantage.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
n_estimators = [1, 3, 5, 10, 15, 20, 40, 70]
random_states = [i for i in range(20)]
criterion = 'entropy' # gini, entropy, log_loss

data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data', header=None, index_col=57)

In [ ]:
for n in n_estimators:
    precision = []
    recall = []
    accuracies = []
    for random_state in random_states:
        # Sample the dataset
        X_train, X_test, y_train, y_test = train_test_split(data, data.index.values, test_size=0.3, random_state=random_state)

        # Fit
        classifier = RandomForestClassifier(n_estimators=n, criterion=criterion, random_state=random_state)
        classifier.fit(X_train, y_train)

        # Predict
        y_pred = classifier.predict(X_test)

        # Evaluate
        accuracies.append(accuracy_score(y_test, y_pred))
        
    accuracies = np.array(accuracies)
    print(f"Random Forest Max Accuracy: {accuracies.max()}\tn_estimator:{n}\tCriterion: {criterion}")

Random Forest Max Accuracy: 0.9232440260680667	n_estimator:1	Criterion: entropy
Random Forest Max Accuracy: 0.9478638667632151	n_estimator:3	Criterion: entropy
Random Forest Max Accuracy: 0.9572773352643013	n_estimator:5	Criterion: entropy
Random Forest Max Accuracy: 0.9630702389572773	n_estimator:10	Criterion: entropy
Random Forest Max Accuracy: 0.9623461259956553	n_estimator:15	Criterion: entropy
Random Forest Max Accuracy: 0.9623461259956553	n_estimator:20	Criterion: entropy
Random Forest Max Accuracy: 0.9688631426502534	n_estimator:40	Criterion: entropy
Random Forest Max Accuracy: 0.9695872556118754	n_estimator:70	Criterion: entropy


## Decision Tree from Scratch

In this part we're going to be implementing a decision tree from scratch. 
We will have following parts: 
1. Reading train and test data. We'll have one `read_data` function for this. 
2. `Node` class. Representing each split in the decision tree. 
3. Implement entropy calculationg and information gain for each tree node.
4. Method to build the tree.


In [ ]:
import math

In [ ]:
# Reading test and train data and dump it to a numpy array
def read_data(filename):
    data = []  # to store training data from csv file
    file_data = []

    # Read data from csv
    with open(filename,"r") as csv_data:
        file_data = csv_data.read().split('\n')

    print(f'File data: {file_data}')
    for i in range(len(file_data)-1):	
        row = file_data[i].split(',')
        for i in range(len(row)):
            row[i] = int(row[i])
        data.append(row)
    data = np.asarray(data)
    return data

In [ ]:
train_data = read_data('data2.csv')
X_train, y_train = train_data[:,:8], train_data[:,8]
X_test = read_data('test2.csv')

File data: ['1,1,1,1,1,1,0,1,1', '1,1,1,1,1,1,0,0,1', '0,1,1,1,1,1,1,1,0', '1,1,1,1,1,0,0,1,1', '1,1,1,1,1,0,0,0,1', '1,1,1,0,1,1,0,1,1', '0,1,0,1,1,1,0,1,0', '1,1,1,0,1,1,0,0,1', '1,1,1,0,1,0,0,1,1', '1,1,1,0,1,0,0,0,1', '0,1,1,1,1,1,0,1,1', '0,1,1,1,1,1,0,0,1', '0,0,1,1,1,1,0,1,0', '0,1,1,1,1,0,0,1,1', '0,1,0,1,0,1,0,1,0', '0,0,0,1,1,1,0,1,0', '0,0,0,1,0,1,1,1,0', '0,1,1,1,1,0,0,0,1', '0,0,1,1,1,1,1,1,0', '0,1,1,0,1,1,0,1,1', '0,0,1,1,0,1,1,1,0', '0,0,0,1,0,1,1,1,0', '1,1,1,0,1,0,1,1,1', '1,1,0,0,1,0,1,1,1', '']
File data: ['0,1,1,1,1,1,1,1', '1,0,0,0,0,0,0,0', '0,1,1,0,1,0,0,0', '0,1,1,1,1,0,0,0', '']


In [ ]:
print(X_train.shape, y_train.shape, X_test.shape)

(24, 8) (24,) (4, 8)


In [ ]:
# Calcualate entropy for those labels
def entropy(labels):
	size = len(labels)
	if size == 0: 
		return 0

	prob_one = 0
	prob_zero = 0

	for i in labels:
		if i == 1:
			prob_one += 1

		elif i == 0:
			prob_zero += 1

	prob_one = float(prob_one)/size
	prob_zero = float(prob_zero)/size

	if prob_zero == 0 or prob_one == 0:
		entropy = 0

	else:
		entropy = -prob_zero*math.log(prob_zero,2) - prob_one*math.log(prob_one,2)

	return entropy

In [ ]:
entropy(y_train)

0.9544340029249649

In [ ]:
# Calculate information gain with a given attribute split
def info_gain(train_data, labels, att_id): # att_id: attribute index to consider
    node_entropy = entropy(labels)
    attributes = train_data[:,att_id]

    # Get all the labels with this attribute being 0
    att_zero = labels[attributes == 0]
    att_zero_count = len(att_zero)

    # Get all the labels with this attribute being 1
    att_one = labels[attributes == 1]
    att_one_count = len(att_one)

    # Calculate the gain
    gain = node_entropy - (float(att_zero_count)/len(labels))*entropy(att_zero) - (float(att_one_count)/len(labels))*entropy(att_one)

    return gain

In [ ]:
# Traverse through the possible attributes and find the attribute that gives
# the maximum information gain
def max_info_gain(train_data, labels):
    idx = -1
    max_gain = -1

    att_idx = [i for i in range(len(train_data[0])) if i not in att_used]
    for att_id in att_idx:
        gain = info_gain(train_data, labels, att_id)
        if gain > max_gain:
            max_gain = gain
            idx = att_id

    return idx

In [ ]:
# Class for one Node
class Node: 
    def __init__(self, train_data, labels): # Gets the attributes and the label given for that node
        self.left = None        # for zero
        self.right = None       # for one

        if entropy(labels) == 0: # Means there is only one label on that node 
            if labels[0] == 1:
                self.att_id = 'Y' # Y means 
            elif labels[0] == 0:
                self.att_id = 'N'	

        else:
            self.att_id = max_info_gain(train_data, labels)	
            att_used.append(self.att_id)

In [ ]:
def create_tree(node, train_data, labels):
	if len(att_used) >= len(train_data[0]): # If all the attributes were already used
		return

	elif node.att_id == 'Y' or node.att_id == 'N':
		return	

	else:	
		left_data = np.reshape(np.extract(np.repeat(np.expand_dims((train_data[:,node.att_id] == 0), axis=1), len(train_data[0]), axis=1), train_data), [-1,train_data.shape[1]])
		left_label = np.reshape(np.extract(train_data[:,node.att_id] == 0, labels), [-1,1]) # Get all the remaining attributes 
		right_data = np.reshape(np.extract(np.repeat(np.expand_dims((train_data[:,node.att_id] == 1), axis=1), len(train_data[0]), axis=1), train_data), [-1,train_data.shape[1]])
		right_label = np.reshape(np.extract(train_data[:,node.att_id] == 1, labels), [-1,1])		

	node.left = Node(left_data, left_label)
	node.right = Node(right_data, right_label)
	create_tree(node.left, left_data, left_label)
	create_tree(node.right, right_data, right_label)

	return node

In [ ]:
def predict(root, test_sample):
	node = root
	while(node.att_id != 'Y' and node.att_id != 'N'):
		if(test_sample[node.att_id] == 0):
			node = node.left
		else: 
			node = node.right
		
	return node.att_id

In [ ]:
att_used = []
root = Node(X_train, y_train)

In [ ]:
root = create_tree(root, X_train, y_train)

In [ ]:
# Get the predictions through test data
output_str = ""

for i in range(len(X_test)):
	if predict(root, X_test[i]) == 'N':
		output_str += str(0) + " "
	else:
		output_str += str(1) + " "

print(output_str)	

0 0 1 1 
